In [ ]:
import pandas as pd
import glob
import os
from sklearn.model_selection import train_test_split
import shutil 
import cv2
import os

### Разделение датасета на train valid

In [ ]:
df = pd.read_csv('./data/labels.csv')

In [ ]:
df_value_counts = df['image'].value_counts() 
df_images = pd.DataFrame({'filename':list(df_value_counts.index),'count': list(df_value_counts)})

In [ ]:
# загружаем картинки без аннотации
images_without_car = list(set([os.path.basename(path) for path in glob.glob('./data/images/*.*')]).difference(set(df_images['filename'].to_list())))
df_images = pd.concat([df_images, pd.DataFrame({'filename':images_without_car, 'count':[0]*len(images_without_car)})])

In [ ]:
# нужно для корректной работы train_test_split
df_images['count'] = df_images['count'].apply(lambda c : 5 if c>5   else c )

In [ ]:
 
os.chdir('./data/images')

In [ ]:

#убираем ломаные изображения 
def check_image(path):
    try:
        cv2.imread(path).shape
        return True
    except Exception :
        return False
df_images = df_images[df_images['filename'].apply(lambda filename :check_image(filename) )]

In [ ]:
df_train, df_valid =  train_test_split(df_images, test_size=0.3, stratify=df_images['count'].values )

In [ ]:
for i,row in df_train.iterrows():
    shutil.copyfile(row['filename'], os.path.join('../yolo_dataset/train/images', row['filename']))

In [ ]:
for i,row in df_valid.iterrows():
    shutil.copyfile(row['filename'],os.path.join('../yolo_dataset/valid/images', row['filename']))

### Перевод разметки в yolo формат

In [ ]:
for i,row in df.iterrows():
     
    try:
        img = cv2.imread(row['image'])
        H,W,_ = img.shape
    except Exception:
        continue
 
    width = (row['xmax']-row['xmin'])/W
    height = (row['ymax']-row['ymin'])/H
    x_center = (row['xmax']+row['xmin'])/2/W
    y_center = (row['ymax']+row['ymin'])/2/H
 
    with open(os.path.join('../yolo_dataset/train/labels/', f"{row['image'].split('.')[0]}.txt"), "a") as file:
        file.write(f"0 {x_center} {y_center} {width} {height} \n")
    

### Обучение модели

In [ ]:
from ultralytics import YOLO
import os 
os.chdir('/home/vlad/projects/Complex_Systems/')

model = YOLO("yolo11n.pt")   
 
results = model.train(data="./data/yolo_dataset/dataset.yaml", epochs=100, imgsz=640)

In [ ]:
model = YOLO("yolo11s.pt")   
 
results = model.train(data="./data/yolo_dataset/dataset.yaml", epochs=100, imgsz=640)